In [0]:
ecom_nov_df = spark.read.csv("/Volumes/idc/idc_kaggle/ecom_data/2019-Nov.csv", header=True, inferSchema=True)
ecom_oct_df = spark.read.csv("/Volumes/idc/idc_kaggle/ecom_data/2019-Oct.csv", header=True, inferSchema=True)

In [0]:
ecom_df = ecom_oct_df.union(ecom_nov_df)

# Convert CSV to Delta format

In [0]:
%sql
create volume if not exists idc.idc_kaggle.ecom_delta_data

In [0]:
%sql
use idc.idc_kaggle

# Create Delta tables (SQL and PySpark)

In [0]:
ecom_df.write \
  .format("delta") \
  .mode("overwrite") \
  .save("/Volumes/idc/idc_kaggle/ecom_delta_data")

In [0]:
ecom_df.write.fornat("delta").saveAsTable("ecom")

In [0]:
%sql
select count(*) from ecom;

In [0]:
%sql
create or replace table ecom_delta
using delta
as select * from ecom;

In [0]:
%sql
select * from ecom_delta;

# Schema Enforcement

In [0]:
try:
  w_schema = spark.createDataFrame([("a","b","c")], ["v","p","m"])
  w_schema.write.mode("overwrite").saveAsTable("ecom")
except Exception as e:
  print("Schema enforcement : ",e)

In [0]:
ecom_oct_df.limit(1000).write.format("delta").mode("overwrite").saveAsTable("ecom_oct")

In [0]:
ecom_nov_df.limit(1000).write.format("delta").mode("overwrite").saveAsTable("ecom_nov")

In [0]:
%sql
select count(*) from ecom_oct;

In [0]:
%sql
MERGE INTO ecom_oct  eo 
using ecom_nov en 
on eo.user_id = en.user_id
when matched then update set *
when not matched then insert *;
    
select count(*) from ecom_oct;